In [1]:
#mosaic an image into a single column and
#save the mosaic
import os.path as path
from shutil import copyfile
import cv2
from opencvlib import view
from opencvlib.view import draw_str
import opencvlib
import opencvlib.imgpipes.generators as gen
import funclib.iolib as iolib
import opencvlib.transforms as tf
import opencvlib.view as view
from funclib.iolib import PrintProgress as PP
print('Done')

Done


In [11]:
#Process Images

In [3]:
src = 'C:/Users/Graham Monkman/OneDrive/ebay/1 tolist'
fp = gen.FromPaths(src)

def resize_(I, fname, width=1920, overwrite_orig=False):
    '''overwrite the original image'''
    I = tf.resize(image=I, width=width)
    if overwrite_orig:
        cv2.imwrite(fname, I)
    return I

n = len(list(1 for _ in fp.generate(pathonly=True)))
P = PP(maximum=n)

for I, fname, _ in fp.generate():
    _, f, ext = iolib.get_file_parts(fname)
    fout = '%s/%s_%s' % (src, f, ext)  #eg. 1.jpg will be 1_.jpg
    I = resize_(I, fname, overwrite_orig=True)
    #Ii = tf.equalize_adapthist(I)
    #DEFAULTS ... h=3, hColor=10, templateWindowSize=7, searchWindowSize=21
    #Ii = cv2.fastNlMeansDenoisingColored(I, None, 10, 10, 7, 21)
    Ii = I
    cv2.imwrite(fout, Ii)
    P.increment()
    
print('Done')



21 of 21 [##############################] 100.0% 

Done


In [2]:
#easy_mosaic
#This should resize images to w=1920, overwriting the originals
#and then create 2x2 mosaics for each image, and delete the originals
#used in the mosaic

cnt = 0

PATCHES = 4
COLS = 2
DO_DELETE = True
DO_RESIZE = False
DO_DENOISE = False
CREATE_BACKUP = False
FLD = 'C:/Users/Graham Monkman/OneDrive/ebay/1 tolist/add_combendium'
FLD = path.normpath(FLD)
FLD_BACKUP = path.normpath('%s/_backup' % FLD)
WIDTH = 2200 #1920
if CREATE_BACKUP:
    iolib.create_folder(FLD_BACKUP)
    fp = gen.FromPaths(FLD)
    for _, fname, _ in fp.generate(pathonly=True):
        d, f, e = iolib.get_file_parts2(fname)
        dst = '%s/%s' % (FLD_BACKUP, f)
        copyfile(fname, dst)
    print('Copied files to %s' % FLD_BACKUP)

    
def resize_(I, fname, width=WIDTH, overwrite_orig=False):
    '''overwrite the original image'''
    I = tf.resize(image=I, width=width)
    if overwrite_orig:
        cv2.imwrite(fname, I)
    return I

if DO_RESIZE or DO_DENOISE:
    fp = gen.FromPaths(FLD)
    n = len(list(1 for _ in fp.generate(pathonly=True)))
    P = PP(maximum=n)
    for I, fname, _ in fp.generate():
        _, f, ext = iolib.get_file_parts(fname)
        if DO_RESIZE:
            I = resize_(I, fname, overwrite_orig=False)
            
        if DO_DENOISE:
            I = cv2.fastNlMeansDenoisingColored(I, None, 10, 10, 7, 21)
    
        cv2.imwrite(fname, I)
        P.increment()
    

files = [f for f in iolib.file_list_generator1(FLD, ['.jpg'], recurse=False)]
P = PP(maximum=len(files))
images = []

for i, f in enumerate(files):
    images.append(f)
    if len(images) == PATCHES:
        I = view.mosaic(images, cols=COLS, save_as='%s/mosaic%s.jpg' % (FLD, int((i + 1)/4)))
        if DO_DELETE:
            iolib.files_delete2(images)
        images = []
    P.increment()
            
if len(images) > 1:
    I = view.mosaic(images, cols=COLS, save_as='%s/mosaic%s.jpg' % (FLD, '_last'))
    if DO_DELETE:
        iolib.files_delete2(images)
    images = []

print('Done')

C:\development\python\opencvlib\view.py:311: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  out = _np.vstack(map(_np.hstack, rows))


18 of 18 [##############################] 100.0% 

Done
